# SandyCo Problem 
Please refer to the material of week 1 of course for more details about this problem.
SandyCo has 2 facilities that supplies materials to three different customer regions.

## Problem Objective
How much sand should SandyCo ship from each plant to each region
each week to meet demand at the lowest planned distribution costs?

## Input Data


In [1]:
S = [100, 125]  # quantity supplied from facility
D = [25, 95, 80]    # quantity demanded at region
C = [
        [250, 325, 445],
        [275, 260, 460]
    ]           # costs from i to j

In [2]:
# defining number of plants and customer's region
num_plants = len(S)
num_regions = len(D)

## Math formulation

In [3]:
#uncomment the line below if you don't have ortools' module installed in your environment and execute this cell
# !pip install ortools

In [4]:
"""
for more information about Google OR-Tools, please refer to https://developers.google.com/optimization
On their website, you will find numerous tutorials and examples that serve as helpful guides.
"""

from ortools.linear_solver import pywraplp

In [5]:
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver('SCIP')


### Decision Variables

In [6]:
# Declare an Matrix to hold our variables for each x_ij.
# When defining the range of the variable, I have already taken into account the constraint x>= 0.
x = {}
for i in range(num_plants):
    for j in range(num_regions):
        x[i,j] = solver.NumVar(0, solver.infinity(), "")

print('Number of variables =', solver.NumVariables())

Number of variables = 6


Note: python start its counting at 0. 

This means we will have x_00 intead of x_11.

### constraints

In [7]:
#Supply limit 
## ex. x11 + x12 + x13 <= S1
for i in range(num_plants):
    sum_value = 0
    for j in range(num_regions):
        sum_value += x[i,j]
    solver.Add(sum_value <= S[i])
    
# Demand requirement
## ex. x11 + x21 >= D1
for j in range(num_regions):
    solver.Add(solver.Sum([x[i,j] for i in range(num_plants)]) >= D[j])

print('Number of constraints =', solver.NumConstraints())

Number of constraints = 5


Note:
The code snippet provided above creates two constraints: supply limit constraints and demand requirement constraints.

For the supply limit constraints, the first approach uses a nested loop to iterate over each plant and region combination, calculating the sum of variables and adding the constraint.
The second approach takes a more pythonic approach, using a list comprehension to generate the variables and calculates the sum.
Despite the difference in implementation, both methods achieve the same result by creating the constraint in the same manner.

For more information about the Pythonic way of coding, you can refer to various resources such as:
https://www.askpython.com/python/examples/pythonic-way-of-coding

### Objective Function

In [8]:
## sum of the cost per ton times the total volume
## min [c11*x11 + c12*c12 + ... ]
solver.Minimize(solver.Sum([C[i][j] * x[i,j] for i in range(num_plants) for j in range(num_regions)]))

### Solve the system.

In [9]:
status = solver.Solve()

In [10]:
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f'Total cost = {solver.Objective().Value()}\n per week')
    for i in range(num_plants):
        for j in range(num_regions):
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            print(f'Plant {i} supplies ' +
                  f'{x[i,j].solution_value()} tons '+
                 f'to Region {j}')
else:
    print('No solution found.')
print('\nAdvanced usage:')
print('Problem solved in %f milliseconds' % solver.wall_time())
print('Problem solved in %d iterations' % solver.iterations())




Total cost = 66625.0
 per week
Plant 0 supplies 25.0 tons to Region 0
Plant 0 supplies 0.0 tons to Region 1
Plant 0 supplies 75.0 tons to Region 2
Plant 1 supplies 0.0 tons to Region 0
Plant 1 supplies 95.0 tons to Region 1
Plant 1 supplies 5.0 tons to Region 2

Advanced usage:
Problem solved in 129.000000 milliseconds
Problem solved in 0 iterations


# Part II - Transhipment Problem
Now, 2 distribution centers are added to the problem. All the demand and supply data remain the same.
## Problem Objective
How much sand should SandyCo ship from each plant to each
packaging center and then to each region each week to meet demand
at the lowest planned distribution costs?

## Input
There are new costs, now that the flow goes from the facility to the DC and from the DC to the region

i -> plants

j -> regions

k -> Distribuiton Center 

In [11]:
C_ik = [
        [190, 210],
        [185, 205]
    ]           # costs from i to k
C_kj = [
    [175, 180, 165],
    [235, 130, 145]
    ]

num_CD = len(C_kj) # or len(C_ik[0]) 

In [12]:
# Create the mip solver with the SCIP.
solverII = pywraplp.Solver.CreateSolver('SCIP')

In [13]:
# Declare our variables for each segment.
# When defining the range of the variables, I have already taken into account the non-negativity constraint.
x = {}   # arcs that go from Plant to CD
y = {}   # arcs that go from CD to region
for i in range(num_plants):
    for k in range(num_CD):
        x[i,k] = solverII.NumVar(0, solverII.infinity(), "")
for k in range(num_CD):
    for j in range(num_regions):
        y[k,j] = solverII.NumVar(0, solverII.infinity(), "")

print('Number of variables =', solverII.NumVariables())

Number of variables = 10


In [14]:
#Supply limit 
for i in range(num_plants):
    solverII.Add(solverII.Sum([x[i,k] for k in range(num_CD)]) <= S[i])
    
# Demand requirement
for j in range(num_regions):
    solverII.Add(solverII.Sum([y[k,j] for k in range(num_CD)]) >= D[j])

# Conservation of flow
for k in range(num_CD):
    solverII.Add(solverII.Sum([x[i, k] for i in range(num_plants)]) -
                 solverII.Sum([y[k, j] for j in range(num_regions)]) ==0)

print('Number of constraints =', solverII.NumConstraints())

Number of constraints = 7


In [15]:
# Define the objective function
objective = solverII.Minimize(
    solverII.Sum([C_ik[i][k] * x[i, k] for i in range(num_plants) for k in range(num_CD)]) +
    solverII.Sum([C_kj[k][j] * y[k, j] for k in range(num_CD) for j in range(num_regions)])
)

In [16]:
status = solverII.Solve()

In [17]:
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f'Total cost = {solverII.Objective().Value()}\n per week')
    for i in range(num_plants):
        for k in range(num_CD):
            print(f'Plant {i} sends ' +
                  f'{x[i,k].solution_value()} tons '+
                 f'to Center {k}')
            
    for k in range(num_CD):
        for j in range(num_regions):
            print(f'CD {k} supplies ' +
                  f'{y[k,j].solution_value()} tons '+
                 f'to Region {j}')
else:
    print('No solution found.')
print('\nAdvanced usage:')
print('Problem solved in %f milliseconds' % solverII.wall_time())
print('Problem solved in %d iterations' % solverII.iterations())



Total cost = 69200.00000000001
 per week
Plant 0 sends 0.0 tons to Center 0
Plant 0 sends 75.0 tons to Center 1
Plant 1 sends 25.0 tons to Center 0
Plant 1 sends 100.0 tons to Center 1
CD 0 supplies 25.0 tons to Region 0
CD 0 supplies 0.0 tons to Region 1
CD 0 supplies 0.0 tons to Region 2
CD 1 supplies 0.0 tons to Region 0
CD 1 supplies 95.0 tons to Region 1
CD 1 supplies 80.0 tons to Region 2

Advanced usage:
Problem solved in 125.000000 milliseconds
Problem solved in 8 iterations
